In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import boto3
import s3fs
import awscli
import timeit
from datetime import datetime
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [2]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

team3-final-bucket
wshih-bucket


In [3]:
my_bucket = s3.Bucket('team3-final-bucket')

In [5]:
client = boto3.client('s3')
path = 's3://team3-final-bucket/sql_db/'

In [6]:
objects_df = pd.read_csv(path + 'objects_clean.csv', dtype={'status':str, 'entity_id':str})
objects_df.head()

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


In [7]:
objects_df.dtypes

Unnamed: 0               int64
object_id               object
entity_type             object
parent_id               object
name                    object
category_code           object
status                  object
founded_at              object
closed_at               object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_total_usd      float64
first_milestone_at      object
last_milestone_at       object
milestones               int64
relationships            int64
dtype: object

In [9]:
is_string_dtype(objects_df['status'])

True

In [10]:
is_numeric_dtype(objects_df['status'])

False

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(objects_df.dtypes)

Unnamed: 0               int64
object_id               object
entity_type             object
parent_id               object
name                    object
category_code           object
status                  object
founded_at              object
closed_at               object
country_code            object
state_code              object
city                    object
region                  object
first_investment_at     object
last_investment_at      object
investment_rounds        int64
invested_companies       int64
first_funding_at        object
last_funding_at         object
funding_rounds           int64
funding_total_usd      float64
first_milestone_at      object
last_milestone_at       object
milestones               int64
relationships            int64
dtype: object


In [12]:
objects_df.head()

,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,...,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,...,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,...,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


##### Converting date columns into date format

In [13]:
# Converting strings to date type
objects_df['founded_at'] = pd.to_datetime(objects_df.founded_at, format='%Y-%m-%d')
objects_df['closed_at'] = pd.to_datetime(objects_df.closed_at, format='%Y-%m-%d')
objects_df['first_investment_at'] = pd.to_datetime(objects_df.first_investment_at, format='%Y-%m-%d')
objects_df['last_investment_at'] = pd.to_datetime(objects_df.last_investment_at, format='%Y-%m-%d')
objects_df['first_funding_at'] = pd.to_datetime(objects_df.first_funding_at, format='%Y-%m-%d')
objects_df['last_funding_at'] = pd.to_datetime(objects_df.last_funding_at, format='%Y-%m-%d')
objects_df['first_milestone_at'] = pd.to_datetime(objects_df.first_milestone_at, format='%Y-%m-%d')
objects_df['last_milestone_at'] = pd.to_datetime(objects_df.last_milestone_at, format='%Y-%m-%d')

In [14]:
objects_df.dtypes

Unnamed: 0                      int64
object_id                      object
entity_type                    object
parent_id                      object
name                           object
category_code                  object
status                         object
founded_at             datetime64[ns]
closed_at              datetime64[ns]
country_code                   object
state_code                     object
city                           object
region                         object
first_investment_at    datetime64[ns]
last_investment_at     datetime64[ns]
investment_rounds               int64
invested_companies              int64
first_funding_at       datetime64[ns]
last_funding_at        datetime64[ns]
funding_rounds                  int64
funding_total_usd             float64
first_milestone_at     datetime64[ns]
last_milestone_at      datetime64[ns]
milestones                      int64
relationships                   int64
dtype: object

In [16]:
# Adding new column for collecting start dates

objects_df["start_dates"] = ""

In [17]:
# Adding a new column for collecting end dates

objects_df["end_dates"] = ""
objects_df.head()

,Unnamed: 0,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,...,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,start_dates,end_dates
0,0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaT,USA,...,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,,
1,1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,USA,...,NaT,NaT,0,0.0,NaT,NaT,0,6,,
2,2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,USA,...,NaT,NaT,0,0.0,2003-02-01,2011-09-23,4,12,,
3,3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,NaN,...,NaT,NaT,0,0.0,NaT,NaT,0,0,,
4,4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,NaN,...,NaT,NaT,0,0.0,NaT,NaT,0,0,,


In [19]:
# Adding a new column for time elapsed

objects_df["time_elapsed"] = ""

In [20]:
objects_df = objects_df[['object_id',
 'entity_type',
 'parent_id',
 'name',
 'category_code',
 'status',
 'start_dates',
 'end_dates',
 'time_elapsed',                        
 'founded_at',
 'first_investment_at',
 'first_funding_at',
 'country_code',
 'state_code',
 'city',
 'region',
 'investment_rounds',
 'invested_companies',
 'funding_rounds',                                
 'closed_at',
 'last_investment_at',
 'last_funding_at',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships']]

In [21]:
objects_df.head()

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,founded_at,...,invested_companies,funding_rounds,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,,,,2005-10-17,...,0,3,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,,,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,,,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [22]:
# Filtering 'companies' only
companies_df = objects_df[objects_df['entity_type']=='Company']
companies_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,founded_at,...,invested_companies,funding_rounds,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,,,,2005-10-17,...,0,3,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,,,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,,,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,,,,2007-01-01,...,0,0,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,,,,2007-11-01,...,0,1,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,,,,1959-01-01,...,0,0,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,,,,2008-07-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,1


##### Working on filling in gap of start and end dates

In [23]:
# Filling out start dates gaps by combining dates from three columns of 'Companies':
# funded_at, first_investment_at, first_funding_at.
# If conflicting dates are found in one record, the earliest is taken.

def mindate(a,b,c):
    return min(a,b,c)

companies_df['start_dates'] = companies_df.apply(lambda row : mindate(row['founded_at'], 
                     row['first_investment_at'], row['first_funding_at']), axis = 1)

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
# Get the max date to fill "end_dates" gap
def maxdate(a,b,c):
    if a == 'NaT':
        return max(b,c)
    elif b == 'NaT':
        return max(a,c)
    else:
        return c
    
companies_df['end_dates'] = companies_df.apply(lambda row : maxdate(row['closed_at'], 
                      row['last_investment_at'], row['last_funding_at']), axis = 1)

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [25]:
companies_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,founded_at,...,invested_companies,funding_rounds,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,2005-10-17,...,0,3,NaT,NaT,2008-05-19,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,NaT,...,0,0,NaT,NaT,NaT,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,2008-07-26,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,2007-01-01,...,0,0,NaT,NaT,NaT,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,2007-11-01,...,0,1,NaT,NaT,2008-08-25,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,1959-01-01,...,0,0,NaT,NaT,NaT,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,2008-07-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,1


In [26]:
# Dropping columns
cleaned_companies_df = companies_df.drop(companies_df[['founded_at','first_investment_at','first_funding_at','closed_at','last_investment_at','last_funding_at']], axis=1)
cleaned_companies_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,USA,...,Culver City,Los Angeles,0,0,0,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,USA,...,San Mateo,SF Bay,0,0,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,USA,...,San Francisco,SF Bay,0,0,0,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,USA,...,Falls Church,Washington DC,0,0,0,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,USA,...,Los Angeles,Los Angeles,0,0,0,0.0,NaT,NaT,0,1


In [27]:
# Dropping nulls in end_dates
cleaned_companies_df[cleaned_companies_df['end_dates'].notna()]

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
13,c:1001,Company,NaN,FriendFeed,web,acquired,2007-10-01,2008-02-26,,USA,...,Mountain View,SF Bay,0,0,1,5000000.0,2008-05-01,2012-09-13,3,14
18,c:10014,Company,NaN,Mobclix,mobile,acquired,2008-03-01,2008-09-01,,USA,...,Palo Alto,SF Bay,0,0,1,0.0,1995-03-01,2012-09-18,4,9
19,c:10015,Company,NaN,Fitbit,health,operating,2007-10-01,2013-08-13,,USA,...,San Francisco,SF Bay,0,0,5,68069200.0,NaT,NaT,0,14
20,c:100155,Company,NaN,MTPV,cleantech,operating,2003-01-01,2012-01-26,,USA,...,Austin,Austin,0,0,3,10125293.0,2010-01-01,2010-01-01,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196537,c:99853,Company,NaN,ChatterBlock,social,operating,2011-01-11,2011-09-06,,CAN,...,Victoria,Victoria,0,0,1,500000.0,2011-01-11,2011-01-11,1,2
196540,c:9989,Company,NaN,Shattered Reality Interactive,games_video,closed,2006-06-01,2009-02-01,,USA,...,San Francisco,SF Bay,0,0,2,1100000.0,2005-11-01,2008-09-10,2,4
196547,c:9994,Company,NaN,Imindi,advertising,operating,2007-01-01,2008-05-01,,NaN,...,NaN,unknown,0,0,2,500000.0,2009-08-27,2009-08-27,1,2
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14


##### There are only 31507 end date records from the companies_df
-----

#### Looking at Product from original df for start and end dates

In [28]:
# Looking at products from the original df for end dates

end_products_df = objects_df[objects_df['entity_type']=='Product']
end_products_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,founded_at,...,invested_companies,funding_rounds,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
434913,r:1,Product,c:1,Wikison Wetpaint,NaN,live,,,,2006-06-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434914,r:10,Product,c:16,Slacker Web Player,NaN,operating,,,,2007-06-05,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434915,r:100,Product,c:1585,Vemo Enterprise Workforce Planning,NaN,live,,,,2008-07-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434916,r:10002,Product,c:14171,dexmo.com - study and revision network,NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434917,r:10003,Product,c:14173,VIrtual PBX Parachute,NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462647,r:9996,Product,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462648,r:9997,Product,c:14164,"Cmail, broadcast email marketing",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462649,r:9998,Product,c:14164,"CatylistCRM, contact database",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [29]:
# Dropping null values in parent_id
cleaned_end_products_df = end_products_df[end_products_df['parent_id'].notnull()]
cleaned_end_products_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed,founded_at,...,invested_companies,funding_rounds,closed_at,last_investment_at,last_funding_at,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
434913,r:1,Product,c:1,Wikison Wetpaint,NaN,live,,,,2006-06-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434914,r:10,Product,c:16,Slacker Web Player,NaN,operating,,,,2007-06-05,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434915,r:100,Product,c:1585,Vemo Enterprise Workforce Planning,NaN,live,,,,2008-07-01,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434916,r:10002,Product,c:14171,dexmo.com - study and revision network,NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
434917,r:10003,Product,c:14173,VIrtual PBX Parachute,NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462647,r:9996,Product,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462648,r:9997,Product,c:14164,"Cmail, broadcast email marketing",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0
462649,r:9998,Product,c:14164,"CatylistCRM, contact database",NaN,operating,,,,NaT,...,0,0,NaT,NaT,NaT,0.0,NaT,NaT,0,0


In [30]:
cleaned_end_products_df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['object_id', 'entity_type', 'parent_id', 'name', 'category_code',
       'status', 'start_dates', 'end_dates', 'time_elapsed', 'founded_at',
       'first_investment_at', 'first_funding_at', 'country_code', 'state_code',
       'city', 'region', 'investment_rounds', 'invested_companies',
       'funding_rounds', 'closed_at', 'last_investment_at', 'last_funding_at',
       'funding_total_usd', 'first_milestone_at', 'last_milestone_at',
       'milestones', 'relationships'],
      dtype='object')>

In [31]:
cleaned_end_products_df['founded_at'].unique()

array(['2006-06-01T00:00:00.000000000', '2007-06-05T00:00:00.000000000',
       '2008-07-01T00:00:00.000000000', ...,
       '2007-03-15T00:00:00.000000000', '2008-11-27T00:00:00.000000000',
       '2007-12-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [32]:
cleaned_end_products_df['first_investment_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [33]:
cleaned_end_products_df['first_funding_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [34]:
# Dropping all redundant columns

ce2_products_df = cleaned_end_products_df.drop(['entity_type',  
       'category_code', 'first_investment_at', 'first_funding_at', 'country_code', 'state_code',
       'city', 'region', 'investment_rounds', 'invested_companies',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at', 'last_milestone_at',
       'milestones', 'relationships'], axis=1)

ce2_products_df

,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at,last_investment_at,last_funding_at
434913,r:1,c:1,Wikison Wetpaint,live,,,,2006-06-01,NaT,NaT,NaT
434914,r:10,c:16,Slacker Web Player,operating,,,,2007-06-05,NaT,NaT,NaT
434915,r:100,c:1585,Vemo Enterprise Workforce Planning,live,,,,2008-07-01,NaT,NaT,NaT
434916,r:10002,c:14171,dexmo.com - study and revision network,operating,,,,NaT,NaT,NaT,NaT
434917,r:10003,c:14173,VIrtual PBX Parachute,operating,,,,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,c:14164,"SiteLink, listing feed for Brokerages",operating,,,,NaT,NaT,NaT,NaT
462647,r:9996,c:14164,"EDCLink, listing feed for Economic Development...",operating,,,,NaT,NaT,NaT,NaT
462648,r:9997,c:14164,"Cmail, broadcast email marketing",operating,,,,NaT,NaT,NaT,NaT
462649,r:9998,c:14164,"CatylistCRM, contact database",operating,,,,NaT,NaT,NaT,NaT


In [35]:
ce2_products_df['last_investment_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [36]:
ce2_products_df['last_funding_at'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [37]:
# Dropping last_investment_at and last_funding_at columns

products_df_final = ce2_products_df.drop(['last_investment_at', 'last_funding_at'], axis=1)
products_df_final

,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at
434913,r:1,c:1,Wikison Wetpaint,live,,,,2006-06-01,NaT
434914,r:10,c:16,Slacker Web Player,operating,,,,2007-06-05,NaT
434915,r:100,c:1585,Vemo Enterprise Workforce Planning,live,,,,2008-07-01,NaT
434916,r:10002,c:14171,dexmo.com - study and revision network,operating,,,,NaT,NaT
434917,r:10003,c:14173,VIrtual PBX Parachute,operating,,,,NaT,NaT
...,...,...,...,...,...,...,...,...,...
462646,r:9995,c:14164,"SiteLink, listing feed for Brokerages",operating,,,,NaT,NaT
462647,r:9996,c:14164,"EDCLink, listing feed for Economic Development...",operating,,,,NaT,NaT
462648,r:9997,c:14164,"Cmail, broadcast email marketing",operating,,,,NaT,NaT
462649,r:9998,c:14164,"CatylistCRM, contact database",operating,,,,NaT,NaT


In [38]:
products_df_final = products_df_final[products_df_final['closed_at'].notna()]
products_df_final

,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at
435039,r:10173,c:14409,Friendzbook,closed,,,,2007-09-21,2007-09-21
435064,r:102,c:460,Sticky Notes,closed,,,,2007-08-18,2010-01-01
435118,r:10253,c:3347,ZuiGames,closed,,,,2008-11-14,2008-11-14
435229,r:10393,c:14624,Bitloot,closed,,,,2008-11-14,2009-04-01
435236,r:104,c:112,Zlio,closed,,,,2006-09-10,2011-09-20
...,...,...,...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,,,,2008-07-22,1993-05-12
461911,r:9055,c:12152,MacFeeder Sverige,closed,,,,2008-07-01,2009-06-01
462096,r:9291,c:12,Twicco,closed,,,,2009-11-04,2009-07-31
462124,r:9320,c:13193,WW.DataServices,closed,,,,2006-04-01,2007-03-01


In [39]:
products_df_final_2 = products_df_final[products_df_final['founded_at'].notna()]
products_df_final_2

,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at
435039,r:10173,c:14409,Friendzbook,closed,,,,2007-09-21,2007-09-21
435064,r:102,c:460,Sticky Notes,closed,,,,2007-08-18,2010-01-01
435118,r:10253,c:3347,ZuiGames,closed,,,,2008-11-14,2008-11-14
435229,r:10393,c:14624,Bitloot,closed,,,,2008-11-14,2009-04-01
435236,r:104,c:112,Zlio,closed,,,,2006-09-10,2011-09-20
...,...,...,...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,,,,2008-07-22,1993-05-12
461911,r:9055,c:12152,MacFeeder Sverige,closed,,,,2008-07-01,2009-06-01
462096,r:9291,c:12,Twicco,closed,,,,2009-11-04,2009-07-31
462124,r:9320,c:13193,WW.DataServices,closed,,,,2006-04-01,2007-03-01


##### Only 151 more start-end_dates pairs would be available from the products_df

In [40]:
# Final product df with start and end dates
products_df_final_2['end_dates'] = products_df_final_2.apply(lambda row : (row['closed_at']), axis = 1)
products_df_final_2

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at
435039,r:10173,c:14409,Friendzbook,closed,,2007-09-21,,2007-09-21,2007-09-21
435064,r:102,c:460,Sticky Notes,closed,,2010-01-01,,2007-08-18,2010-01-01
435118,r:10253,c:3347,ZuiGames,closed,,2008-11-14,,2008-11-14,2008-11-14
435229,r:10393,c:14624,Bitloot,closed,,2009-04-01,,2008-11-14,2009-04-01
435236,r:104,c:112,Zlio,closed,,2011-09-20,,2006-09-10,2011-09-20
...,...,...,...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,,1993-05-12,,2008-07-22,1993-05-12
461911,r:9055,c:12152,MacFeeder Sverige,closed,,2009-06-01,,2008-07-01,2009-06-01
462096,r:9291,c:12,Twicco,closed,,2009-07-31,,2009-11-04,2009-07-31
462124,r:9320,c:13193,WW.DataServices,closed,,2007-03-01,,2006-04-01,2007-03-01


In [41]:
# Final product df with start and end dates
products_df_final_2['start_dates'] = products_df_final_2.apply(lambda row : (row['founded_at']), axis = 1)
products_df_final_2['end_dates'] = products_df_final_2.apply(lambda row : (row['closed_at']), axis = 1)
products_df_final_2

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed,founded_at,closed_at
435039,r:10173,c:14409,Friendzbook,closed,2007-09-21,2007-09-21,,2007-09-21,2007-09-21
435064,r:102,c:460,Sticky Notes,closed,2007-08-18,2010-01-01,,2007-08-18,2010-01-01
435118,r:10253,c:3347,ZuiGames,closed,2008-11-14,2008-11-14,,2008-11-14,2008-11-14
435229,r:10393,c:14624,Bitloot,closed,2008-11-14,2009-04-01,,2008-11-14,2009-04-01
435236,r:104,c:112,Zlio,closed,2006-09-10,2011-09-20,,2006-09-10,2011-09-20
...,...,...,...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,2008-07-22,1993-05-12,,2008-07-22,1993-05-12
461911,r:9055,c:12152,MacFeeder Sverige,closed,2008-07-01,2009-06-01,,2008-07-01,2009-06-01
462096,r:9291,c:12,Twicco,closed,2009-11-04,2009-07-31,,2009-11-04,2009-07-31
462124,r:9320,c:13193,WW.DataServices,closed,2006-04-01,2007-03-01,,2006-04-01,2007-03-01


In [42]:
#Dropping columns fp = final product
fp_df = products_df_final_2.drop(products_df_final_2[['closed_at', 'founded_at']], axis=1)
fp_df

,object_id,parent_id,name,status,start_dates,end_dates,time_elapsed
435039,r:10173,c:14409,Friendzbook,closed,2007-09-21,2007-09-21,
435064,r:102,c:460,Sticky Notes,closed,2007-08-18,2010-01-01,
435118,r:10253,c:3347,ZuiGames,closed,2008-11-14,2008-11-14,
435229,r:10393,c:14624,Bitloot,closed,2008-11-14,2009-04-01,
435236,r:104,c:112,Zlio,closed,2006-09-10,2011-09-20,
...,...,...,...,...,...,...,...
461825,r:8963,c:11424,PlayCrafter,closed,2008-07-22,1993-05-12,
461911,r:9055,c:12152,MacFeeder Sverige,closed,2008-07-01,2009-06-01,
462096,r:9291,c:12,Twicco,closed,2009-11-04,2009-07-31,
462124,r:9320,c:13193,WW.DataServices,closed,2006-04-01,2007-03-01,


In [121]:
merged_df = pd.merge(cleaned_companies_df, fp_df, how='left', on=['object_id'])
merged_df

,object_id,entity_type,parent_id_x,name_x,category_code,status_x,start_dates_x,end_dates_x,time_elapsed_x,country_code,...,first_milestone_at,last_milestone_at,milestones,relationships,parent_id_y,name_y,status_y,start_dates_y,end_dates_y,time_elapsed_y
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,2010-09-05,2013-09-18,5,17,NaN,NaN,NaN,NaT,NaT,NaN
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,USA,...,NaT,NaT,0,6,NaN,NaN,NaN,NaT,NaT,NaN
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,USA,...,2003-02-01,2011-09-23,4,12,NaN,NaN,NaN,NaT,NaT,NaN
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,NaN,...,NaT,NaT,0,0,NaN,NaN,NaN,NaT,NaT,NaN
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,NaN,...,NaT,NaT,0,0,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,USA,...,2013-08-12,2013-08-12,2,5,NaN,NaN,NaN,NaT,NaT,NaN
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,2013-05-01,2013-05-01,1,14,NaN,NaN,NaN,NaT,NaT,NaN
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,USA,...,2012-01-01,2013-10-30,3,44,NaN,NaN,NaN,NaT,NaT,NaN
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,USA,...,NaT,NaT,0,1,NaN,NaN,NaN,NaT,NaT,NaN


In [122]:
merged_df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['object_id', 'entity_type', 'parent_id_x', 'name_x', 'category_code',
       'status_x', 'start_dates_x', 'end_dates_x', 'time_elapsed_x',
       'country_code', 'state_code', 'city', 'region', 'investment_rounds',
       'invested_companies', 'funding_rounds', 'funding_total_usd',
       'first_milestone_at', 'last_milestone_at', 'milestones',
       'relationships', 'parent_id_y', 'name_y', 'status_y', 'start_dates_y',
       'end_dates_y', 'time_elapsed_y'],
      dtype='object')>

In [123]:
# checking to see if there are any not NaT values
merged_df['start_dates_y'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [124]:
# checking for any unique values
merged_df['end_dates_y'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [125]:
# Dropping all redundant columns

clean_merge_df = merged_df.drop(['parent_id_y',
       'name_y', 'status_y', 'start_dates_y', 'end_dates_y', 'time_elapsed_y'], axis=1)

clean_merge_df

,object_id,entity_type,parent_id_x,name_x,category_code,status_x,start_dates_x,end_dates_x,time_elapsed_x,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,USA,...,Culver City,Los Angeles,0,0,0,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,USA,...,San Mateo,SF Bay,0,0,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,USA,...,San Francisco,SF Bay,0,0,0,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,USA,...,Falls Church,Washington DC,0,0,0,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,USA,...,Los Angeles,Los Angeles,0,0,0,0.0,NaT,NaT,0,1


In [126]:
clean_merge_df = clean_merge_df.rename(columns = {'parent_id_x': 'parent_id', 
                'name_x':'name', 'status_x':'status', 'start_dates_x':'start_dates',
                'end_dates_x':'end_dates', 'time_elapsed_x':'time_elapsed_months'})

clean_merge_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed_months,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,USA,...,Culver City,Los Angeles,0,0,0,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,USA,...,San Mateo,SF Bay,0,0,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,USA,...,San Francisco,SF Bay,0,0,0,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,USA,...,Falls Church,Washington DC,0,0,0,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,USA,...,Los Angeles,Los Angeles,0,0,0,0.0,NaT,NaT,0,1


In [127]:
clean_merge_df.dtypes

object_id                      object
entity_type                    object
parent_id                      object
name                           object
category_code                  object
status                         object
start_dates            datetime64[ns]
end_dates              datetime64[ns]
time_elapsed_months            object
country_code                   object
state_code                     object
city                           object
region                         object
investment_rounds               int64
invested_companies              int64
funding_rounds                  int64
funding_total_usd             float64
first_milestone_at     datetime64[ns]
last_milestone_at      datetime64[ns]
milestones                      int64
relationships                   int64
dtype: object

In [128]:
clean_merge_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed_months,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaT,NaT,,USA,...,Culver City,Los Angeles,0,0,0,0.0,NaT,NaT,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaT,NaT,,USA,...,San Mateo,SF Bay,0,0,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaT,,NaN,...,NaN,unknown,0,0,0,0.0,NaT,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196548,c:99940,Company,NaN,Webgility,ecommerce,operating,2007-01-01,NaT,,USA,...,San Francisco,SF Bay,0,0,0,0.0,2013-08-12,2013-08-12,2,5
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14
196550,c:9996,Company,NaN,CSC,consulting,operating,1959-01-01,NaT,,USA,...,Falls Church,Washington DC,0,0,0,0.0,2012-01-01,2013-10-30,3,44
196551,c:9997,Company,NaN,Top-candidate,search,operating,2008-07-01,NaT,,USA,...,Los Angeles,Los Angeles,0,0,0,0.0,NaT,NaT,0,1


In [129]:
clean_merge_df = clean_merge_df[clean_merge_df['start_dates'].notna() & clean_merge_df['end_dates'].notna()] 
clean_merge_df

,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed_months,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
13,c:1001,Company,NaN,FriendFeed,web,acquired,2007-10-01,2008-02-26,,USA,...,Mountain View,SF Bay,0,0,1,5000000.0,2008-05-01,2012-09-13,3,14
18,c:10014,Company,NaN,Mobclix,mobile,acquired,2008-03-01,2008-09-01,,USA,...,Palo Alto,SF Bay,0,0,1,0.0,1995-03-01,2012-09-18,4,9
19,c:10015,Company,NaN,Fitbit,health,operating,2007-10-01,2013-08-13,,USA,...,San Francisco,SF Bay,0,0,5,68069200.0,NaT,NaT,0,14
20,c:100155,Company,NaN,MTPV,cleantech,operating,2003-01-01,2012-01-26,,USA,...,Austin,Austin,0,0,3,10125293.0,2010-01-01,2010-01-01,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196537,c:99853,Company,NaN,ChatterBlock,social,operating,2011-01-11,2011-09-06,,CAN,...,Victoria,Victoria,0,0,1,500000.0,2011-01-11,2011-01-11,1,2
196540,c:9989,Company,NaN,Shattered Reality Interactive,games_video,closed,2006-06-01,2009-02-01,,USA,...,San Francisco,SF Bay,0,0,2,1100000.0,2005-11-01,2008-09-10,2,4
196547,c:9994,Company,NaN,Imindi,advertising,operating,2007-01-01,2008-05-01,,NaN,...,NaN,unknown,0,0,2,500000.0,2009-08-27,2009-08-27,1,2
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14


In [131]:
# find time elapsed in months 
date1 = clean_merge_df['start_dates'].iloc[0:]
date2 = clean_merge_df['end_dates'].iloc[0:]

clean_merge_df['time_elapsed_months'] = ((date2 - date1)/np.timedelta64(1, 'M')).round()

clean_merge_df

C:\Users\Fahima Farha PC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,object_id,entity_type,parent_id,name,category_code,status,start_dates,end_dates,time_elapsed_months,country_code,...,city,region,investment_rounds,invested_companies,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-01,2008-05-19,32.0,USA,...,Seattle,Seattle,0,0,3,39750000.0,2010-09-05,2013-09-18,5,17
13,c:1001,Company,NaN,FriendFeed,web,acquired,2007-10-01,2008-02-26,5.0,USA,...,Mountain View,SF Bay,0,0,1,5000000.0,2008-05-01,2012-09-13,3,14
18,c:10014,Company,NaN,Mobclix,mobile,acquired,2008-03-01,2008-09-01,6.0,USA,...,Palo Alto,SF Bay,0,0,1,0.0,1995-03-01,2012-09-18,4,9
19,c:10015,Company,NaN,Fitbit,health,operating,2007-10-01,2013-08-13,70.0,USA,...,San Francisco,SF Bay,0,0,5,68069200.0,NaT,NaT,0,14
20,c:100155,Company,NaN,MTPV,cleantech,operating,2003-01-01,2012-01-26,109.0,USA,...,Austin,Austin,0,0,3,10125293.0,2010-01-01,2010-01-01,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196537,c:99853,Company,NaN,ChatterBlock,social,operating,2011-01-11,2011-09-06,8.0,CAN,...,Victoria,Victoria,0,0,1,500000.0,2011-01-11,2011-01-11,1,2
196540,c:9989,Company,NaN,Shattered Reality Interactive,games_video,closed,2006-06-01,2009-02-01,32.0,USA,...,San Francisco,SF Bay,0,0,2,1100000.0,2005-11-01,2008-09-10,2,4
196547,c:9994,Company,NaN,Imindi,advertising,operating,2007-01-01,2008-05-01,16.0,NaN,...,NaN,unknown,0,0,2,500000.0,2009-08-27,2009-08-27,1,2
196549,c:9995,Company,NaN,FohBoh,public_relations,operating,2007-11-01,2008-08-25,10.0,USA,...,San Jose,SF Bay,0,0,1,750000.0,2013-05-01,2013-05-01,1,14


In [153]:
path2 = '../FWIP_Venture/'

In [159]:
time_elapsed_clean = clean_merge_df

time_elapsed_clean.to_csv(path2 + "time_elapsed_clean.csv", index=False)

In [162]:
filename = 'time_elapsed_clean.csv'
bucket = 'team3-final-bucket'

s3.meta.client.upload_file(Filename = filename, Bucket= bucket, Key = filename)